# Desafío 5: Carlos Rubio

Ejercicio 1: Trabajaremos con 3 archivos y manipularemos los datos

In [1]:
#Cargamos las librerías a utilizar
import pandas as pd
import numpy as np

In [2]:
#Creamos los DataFrames de los tres archivos
incidents = pd.read_pickle('incidents.pkl')
officers = pd.read_pickle('officers.pkl')
subjects = pd.read_pickle('subjects.pkl')

In [3]:
#Creamos un DataFrame con la unión de los 3 archivos, usamos el atributo pd.merge(), la unión será tipo 'outer' para no perder ningún dato en la unión
#como existen columnas en comun en los DataFrames officers y subjects, usamos la opción suffixes para diferenciarlos
df = pd.merge(incidents,pd.merge(officers,subjects,on= 'case_number',suffixes=['_officer','_subject'],how='outer'),on= 'case_number',how='outer')

In [4]:
#Si existen filas duplicadas, se eliminan en el nuevo DataFrame df_sin_duplicados
df_sin_duplicados=df.set_index(df.case_number).drop_duplicates()

In [8]:
#Contamos cuantos sujetos femeninos existen en los datos. Pueden ser oficiales o sospechosas.
mujeres=df_sin_duplicados.value_counts('gender_officer')+df_sin_duplicados.value_counts('gender_subject')
print(f'Hay {mujeres.iloc[1]} sujetos femeninos en los datos')

Hay 29 sujetos femeninos en los datos


In [9]:
#Contamos cuantos casos hay por lo menos una mujer sospechosa. Para eso filtramos por el género de las sospechosas y luego usamos el atributo .unique() para descartar duplicados en los casos.
#La función len() cuenta el número de casos que se obtienen.
casos=len(df_sin_duplicados[df_sin_duplicados.gender_subject=='F'].case_number.unique())
print(f'Hay {casos} casos donde hay por lo menos una mujer sospechosa')

Hay 7 casos donde hay por lo menos una mujer sospechosa


In [10]:
#Creamos la tabla pivote entre el género de los oficiales y los sospechosos, la función de agregación es 'count' para contar el número de veces que se relacionan
df_pivot = pd.pivot_table(df_sin_duplicados,values='case_number',index = 'gender_officer',columns='gender_subject',aggfunc= 'count',margins=True)
df_pivot

gender_subject,F,M,All
gender_officer,,,
F,2,18,20
M,7,350,357
All,9,368,377


De la tabla se observa que en su mayoría fueron oficiales hombres los involucrados con sospechosos hombres. Además el total de mujeres sospechosas es 9, eso quiere decir que o en un caso hubo implicada tres mujeres o en dos casos fueron dos sospechosas (que es el caso de esta muestra).

Ejercicio 2

In [11]:
#Cargamos el nuevo archivo
df = pd.read_csv('Cleaned_DS_Jobs.csv')

In [12]:
#Reemplazamos los valores considerados nulos por np.nan
nulos=["na", "NA",-1, "0", "-1", "null", "n/a", "N/A", "NULL"]
df=df.replace(nulos,np.nan)

In [13]:
#eliminamos las filas con datos faltantes
df=df.dropna()

In [14]:
#Creamos las columnas 'Salario Estimado Mínimo' y 'Salario Estimado Máximo'. 
#Usamos el método apply con una función lambda que separa el string de la columna 'Salary Estimate' por el '-'
df[['Salario Estimado Mínimo', 'Salario Estimado Máximo']] = \
df['Salary Estimate'].apply(lambda x: pd.Series([i for i in x.split('-')]))

In [15]:
#Cremos un diccionario donde se califica el tipo de empresa según el tamaño que tiene
tipo_empresa=['Mega Empresas','Grandes Empresas','Medianas Empresas','Pequeñas Empresas',\
              'Pequeñas Grandes Empresas','Microempresas','Empresas sin Información']
tamaño_empresa=dict(zip(sorted(df.Size.unique()),tipo_empresa))
#Reemplazamos la columna 'Size' por la clasificación del tipo de empresa usando el diccionario.
df_replaced = df.replace(tamaño_empresa).reset_index(drop=True)

In [16]:
#Transformamos los valores de las columnas de Salario de string a float
df_replaced[['Salario Estimado Máximo','Salario Estimado Mínimo']]=\
df_replaced[['Salario Estimado Máximo','Salario Estimado Mínimo']].apply(pd.to_numeric)

#Creamos una tabla pivote con la media de los Salarios en función del tamaño de la empresa
df_pivot = pd.pivot_table(df_replaced,values=['Salario Estimado Mínimo','Salario Estimado Máximo'],\
                          index = 'Size',aggfunc= 'mean',margins=True)

#Renombramos las columnas
df_pivot = df_pivot.rename(columns={'Salario Estimado Mínimo':'Promedio Salario Mínimo','Salario Estimado Máximo':'Promedio Salario Máximo'})

df_pivot

,Promedio Salario Máximo,Promedio Salario Mínimo
Size,,
Empresas sin Información,110.500000,73.000000
Grandes Empresas,137.461538,93.923077
Medianas Empresas,141.142857,93.571429
Mega Empresas,151.111111,97.888889
Microempresas,137.666667,100.666667
Pequeñas Empresas,138.875000,92.125000
Pequeñas Grandes Empresas,146.235294,100.176471
All,141.137500,95.512500
